In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

from MultiTimeframeCandleManager import *
from datetime import datetime, timedelta
import time
from collections import deque
import numpy as np
import copy
import tensorflow as tf
from tqdm import tqdm
import random
from save_and_load import *
from Candle import Candle
import matplotlib.pyplot as plt
from tensorflow.keras.layers import (
        Input, Lambda, Concatenate, Dense, Embedding, Dropout, LSTM, 
        MultiHeadAttention, LayerNormalization, LeakyReLU, GlobalAveragePooling1D, Add
    )
from tensorflow.keras import backend as K


#candles = obj_load("/content/NQ_2")[-100000:]
candles = obj_load("../candle_data/NQ_1")[-20000:]
len(candles)

2025-08-14 08:23:39.228042: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-14 08:23:39.256604: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX_VNNI, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-14 08:23:39.822028: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


loading ../candle_data/NQ_1


20000

In [2]:
import tensorflow as tf
print("TF version:", tf.__version__)
print("CUDA version TF was built with:", tf.sysconfig.get_build_info()["cuda_version"])
print("cuDNN version TF was built with:", tf.sysconfig.get_build_info()["cudnn_version"])

TF version: 2.20.0-dev0+selfbuilt
CUDA version TF was built with: 12.8.1
cuDNN version TF was built with: 9


In [3]:
def make_model():
    l = 480
    
    def process_chart_with_time_position(chart_input, name):
        # Extract OHLC and clip values
        ohlc = Lambda(lambda x: tf.clip_by_value(x[:, :, :4], -1000.0, 1000.0),
                      name=f'{name}_clip_ohlc')(chart_input)
        
        # Extract time column and cast to int
        t = Lambda(lambda x: x[:, :, 4:5], name=f'{name}_extract_time')(chart_input)
        t_int = Lambda(lambda x: tf.cast(tf.squeeze(x, axis=-1), tf.int32),
                       name=f'{name}_cast_time')(t)
    
        # Time embedding
        time_embed_layer = Embedding(input_dim=24*60, output_dim=8, name=f'{name}_time_embed')
        t_embed = time_embed_layer(t_int)
    
        # Position embedding (based on sequence length from chart_input shape)
        seq_length = chart_input.shape[1]
        positions = tf.range(seq_length)
        position_embed = Embedding(input_dim=seq_length, output_dim=8,
                                   name=f'{name}_pos_embed')(positions)

        # Add batch dimension so shape matches (batch, seq_length, 8)
        position_embed = tf.expand_dims(position_embed, axis=0)  # (1, 480, 8)
        position_embed = Lambda(lambda x: K.tile(x[0], [K.shape(x[1])[0], 1, 1]))([position_embed, chart_input])
        
        # Combine time and position embeddings
        enhanced_time = Add(name=f'{name}_add_time_pos')([t_embed, position_embed])
    
        # Concatenate OHLC(4) + enhanced_time(8) → 12 dims
        return Concatenate(name=f'{name}_concat')([ohlc, enhanced_time])

    
    def relational_attention_block(x, num_heads=8, key_dim=64, name_prefix=""):
        """Self-attention to capture relationships between candles"""
        # Multi-head self-attention
        attention_out = MultiHeadAttention(
            num_heads=num_heads, 
            key_dim=key_dim,
            name=f'{name_prefix}_attention'
        )(x, x)
        
        # Residual connection + layer norm
        x_normed = LayerNormalization(name=f'{name_prefix}_norm1')(x + attention_out)
        
        # Feed-forward network
        ff_dim = x.shape[-1] * 2
        ff = Dense(ff_dim, activation='gelu', name=f'{name_prefix}_ff1')(x_normed)
        ff = Dense(x.shape[-1], name=f'{name_prefix}_ff2')(ff)
        
        # Second residual connection + layer norm
        output = LayerNormalization(name=f'{name_prefix}_norm2')(x_normed + ff)
        
        return output
    
    def process_timeframe_with_attention(chart_input, pdas_repeated, name):
        """Process each timeframe with attention mechanisms"""
        # Concatenate chart with PDAs
        x = Concatenate(axis=-1, name=f'{name}_concat_pdas')([chart_input, pdas_repeated])
        
        # Initial feature extraction - reduce dimensions gradually
        x = Dense(512, name=f'{name}_dense1')(x)
        x = LayerNormalization(name=f'{name}_norm_init')(x)
        x = LeakyReLU(0.1, name=f'{name}_relu1')(x)
        x = Dropout(0.1, name=f'{name}_dropout1')(x)
        
        x = Dense(256, name=f'{name}_dense2')(x)
        x = LayerNormalization(name=f'{name}_norm2')(x)
        x = LeakyReLU(0.1, name=f'{name}_relu2')(x)
        
        # Apply attention blocks to capture relational patterns
        x = relational_attention_block(x, num_heads=8, key_dim=64, name_prefix=f'{name}_attn1')
        x = relational_attention_block(x, num_heads=8, key_dim=64, name_prefix=f'{name}_attn2')
        x = relational_attention_block(x, num_heads=8, key_dim=64, name_prefix=f'{name}_attn3')
        x = relational_attention_block(x, num_heads=8, key_dim=64, name_prefix=f'{name}_attn4')
        x = relational_attention_block(x, num_heads=8, key_dim=64, name_prefix=f'{name}_attn5')
        x = relational_attention_block(x, num_heads=8, key_dim=64, name_prefix=f'{name}_attn6')
        
        # Final sequence compression with LSTM (single layer is often enough after attention)
        x = LSTM(256, return_sequences=False, name=f'{name}_lstm')(x)
        
        return x
    
    
    ### Inputs
    input_chart_m15 = Input(shape=(l,5), name='chart_m15')
    input_chart_m5  = Input(shape=(l,5), name='chart_m5')
    input_chart_m1  = Input(shape=(l,5), name='chart_m1')
    
    pdas = Input(shape=(3*3 + 3*3 + 1 + 12*5 + 5*3,), name='pdas')
    pdas = Lambda(lambda x: tf.clip_by_value(x, -1000.0, 1000.0), name='clip_pdas')(pdas)
    
    #minutes = Input(shape=(1,), name='minutes')
    
    ### Process each chart with enhanced time/position embeddings
    chart_m15 = process_chart_with_time_position(input_chart_m15, 'm15')
    chart_m5  = process_chart_with_time_position(input_chart_m5, 'm5')
    chart_m1  = process_chart_with_time_position(input_chart_m1, 'm1')
    
    # Repeat PDAs for concatenation
    pdas_repeated = Lambda(
        lambda inputs: tf.repeat(tf.expand_dims(inputs, axis=1), repeats=l, axis=1),
        name='repeat_pdas'
    )(pdas)
    
    ### Process each timeframe with attention
    m15_features = process_timeframe_with_attention(chart_m15, pdas_repeated, 'm15')
    m5_features = process_timeframe_with_attention(chart_m5, pdas_repeated, 'm5') 
    m1_features = process_timeframe_with_attention(chart_m1, pdas_repeated, 'm1')

    ### Final combination and prediction
    # Combine all enhanced features with original PDAs
    c = Concatenate(name='final_concat')([pdas, m1_features, m5_features, m15_features])
    
    # Final prediction layers - smaller than before since attention does heavy lifting
    d = Dense(2048, name='pred_dense1')(c)
    d = LayerNormalization(name='pred_norm1')(d)
    d = LeakyReLU(0.1, name='pred_relu1')(d)
    d = Dropout(0.1, name='pred_dropout1')(d)
    
    d = Dense(1024, name='pred_dense2')(d)
    d = LayerNormalization(name='pred_norm2')(d)
    d = LeakyReLU(0.1, name='pred_relu2')(d)
    d = Dropout(0.1, name='pred_dropout2')(d)
    
    d = Dense(512, name='pred_dense3')(d)
    d = LayerNormalization(name='pred_norm3')(d)
    d = LeakyReLU(0.1, name='pred_relu3')(d)
    
    # Stabilization layer before final prediction
    d = Dense(256, activation='tanh', name='pre_softmax_tanh')(d)
    
    # Final prediction
    output = Dense(3, activation="softmax", dtype="float32", name='output')(d)
    
    model = tf.keras.Model(inputs=[input_chart_m15, input_chart_m5, input_chart_m1, pdas], 
                  outputs=output)
    
    return model

In [4]:
model = make_model()

2025-08-14 08:23:40.266885: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-08-14 08:23:40.266899: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:160] env: CUDA_VISIBLE_DEVICES="-1"
2025-08-14 08:23:40.266901: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] CUDA_VISIBLE_DEVICES is set to -1 - this hides all GPUs from CUDA
2025-08-14 08:23:40.266903: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:171] verbose logging is disabled. Rerun with verbose logging (usually --v=1 or --vmodule=cuda_diagnostics=1) to get more diagnostic output from this module
2025-08-14 08:23:40.266905: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:176] retrieving CUDA diagnostic information for host: rtx5090server
2025-08-14 08:23:40.266906: I external/local_xla/xla/stream_executor/cuda

In [5]:
model.load_weights("model.weights.h5")

In [6]:
class Order:
    def __init__(self, limit, stop, tp, direction):
        self.entry = limit
        self.tp = tp
        self.sl = stop
        self.direction = direction

class Position:
    def __init__(self, entry, stop, tp, direction):
        self.entry = entry
        self.tp = tp
        self.sl = stop
        self.direction = direction

In [7]:
def step(index):

        global current_position, current_order, slm, m, outputs, all_candles, cmm, equity, equity_L
    
        ret = m.push_m1_candle(candles[index])
        midnight_open, midnight_opening_range_high,midnight_opening_range_low, pdas, current_close, current_time, charts = ret
        center = (midnight_opening_range_high + midnight_opening_range_low) / 2
        r = max(0.0001, (midnight_opening_range_high - midnight_opening_range_low) / 2)



        current_candle_m1 = charts[2][-1]
        #### check tp before filling order so that the same m1 candle will not trigger tp - it is not sure if the candle hit first limit and later tp or reve3rse
        if current_position.direction == 1:
            if current_candle_m1.h >= current_position.tp:
                pnl = (current_position.tp - current_position.entry) * current_position.direction
                equity += pnl
                current_position = Position(0,0,0,0)
        if current_position.direction == -1:
            if current_candle_m1.l <= current_position.tp:
                pnl = (current_position.tp - current_position.entry) * current_position.direction
                equity += pnl
                current_position = Position(0,0,0,0)

        #### check order
        if current_order != None:
            if  current_order.direction == 1:
                if current_candle_m1.l < current_order.entry:
                    current_position = Position(current_order.entry, current_order.sl, current_order.tp, current_order.direction)
                    #print("fill long order:",current_order.entry, current_order.sl, current_order.tp)
                    equity -= cmm
                    current_order = None
        if current_order != None:
            if  current_order.direction == -1:
                if current_candle_m1.h > current_order.entry:
                    current_position = Position(current_order.entry, current_order.sl, current_order.tp, current_order.direction)
                    #print("fill short order:",current_order.entry, current_order.sl, current_order.tp)
                    equity -= cmm
                    current_order = None

        #### check sl
        if current_position.direction == 1:
            if current_candle_m1.l <= current_position.sl:
                pnl = (current_position.sl - current_position.entry) * current_position.direction
                equity += pnl
                current_position = Position(0,0,0,0)
        if current_position.direction == -1:
            if current_candle_m1.h >= current_position.sl:
                pnl = (current_position.sl - current_position.entry) * current_position.direction
                equity += pnl
                current_position = Position(0,0,0,0)




        if(len(m.m15_candles) == 480):


            open_profit = (current_close - current_position.entry) * current_position.direction

            scaled_entry_diff  =  0
            scaled_sl_diff  =  0
            if(current_position.direction != 0):
                scaled_entry_diff = (current_close - current_position.entry) / r
                scaled_sl_diff = (current_close - current_position.sl) / r

            state = ret_to_scaled_inputs(ret) + [np.array([current_position.direction, scaled_entry_diff, scaled_sl_diff])]
            m15_np, m5_np, m1_np, pda_np, pos_info = state

            equity_L.append(equity+open_profit)
            all_candles.append(charts[2][-1])


            if True:#current_minutes >= 9*60+29 and current_minutes < 16*60:
                output = inference_step(
                    tf.expand_dims(m15_np, 0),
                    tf.expand_dims(m5_np, 0),
                    tf.expand_dims(m1_np, 0),
                    tf.expand_dims(pda_np, 0),
                    #tf.expand_dims(pos_info, 0)
                )

              
                last_action = np.argmax(output)
                #last_action = np.argmax([output[0][0], output[0][1]])
                outputs.append(output[0])
            else:
                last_action = 2


            avg_candle_range = np.mean([ i.h - i.l for i in list(charts[2])])

            if(last_action == 2):
                #equity += open_profit
                #current_position = Position(0,0,0,0)
                #print("close position:", open_profit)
                current_order = None

            if(last_action == 0 and current_position.direction == 1):
                equity += open_profit
                current_position = Position(0,0,0,0)

            if(last_action == 0):
                last_candle_low = charts[2][-2].l
                if ( last_candle_low < current_close ):
                    last_candle_low = None

                pdas = m.normal_pdas ## (low, high)

                ## ignore pdas with low below close
                pdas_filtered = []
                for pda in pdas:
                        if(pda[0] > current_close):
                            pdas_filtered.append(pda)
                ### sort
                sorted_by_high = sorted(pdas_filtered, key = lambda x:x[1])
                sorted_by_low = sorted(pdas_filtered, key = lambda x:x[0])

                if(len(pdas_filtered) > 2):

                    ### entry is lowest i can get or immediate rebalance
                    entry = sorted_by_low[0][0]
                    if(last_candle_low != None):
                        entry = min(entry, last_candle_low)


                    #sl = entry + avg_candle_range * slm
                    sl = sorted_by_low[2][1]
                    tp = entry  -  avg_candle_range * tpm

                    if current_position.direction == 0:
                        current_order = Order(entry, sl, tp, -1)
                        #print("set short order:",entry,sl,tp)
                    if current_position.direction == -1:
                        #current_position.sl = sl
                        current_position.tp = tp



            if(last_action == 1 and current_position.direction == -1):
                equity += open_profit
                current_position = Position(0,0,0,0)

            if(last_action == 1):
                last_candle_high = charts[2][-2].h
                if ( last_candle_high > current_close ):
                    last_candle_high = None
                pdas = m.normal_pdas ## (low, high)

                ## ignore pdas with low below close
                pdas_filtered = []
                for pda in pdas:
                        if(pda[1] < current_close):
                            pdas_filtered.append(pda)
                ### sort
                sorted_by_high = sorted(pdas_filtered, key = lambda x:x[1], reverse=True)
                sorted_by_low = sorted(pdas_filtered, key = lambda x:x[0], reverse=True)

                if(len(pdas_filtered) > 2):
                    ### entry is lowest i can get or immediate rebalance
                    entry = sorted_by_high[0][1]
                    if(last_candle_high != None):
                        entry = max(entry, last_candle_high)

                    #sl = entry - avg_candle_range * slm
                    sl = sorted_by_high[2][0]
                    tp = entry  +  avg_candle_range * tpm

                    if current_position.direction == 0:
                        current_order = Order(entry, sl, tp, 1)
                        #print("set long order:",entry,sl,tp)
                    if current_position.direction == 1:
                        #current_position.sl = sl
                        current_position.tp = tp

In [ ]:
m = MultiTimeframeCandleManager()

#slm = 2
#tpm = 6

slm = 2
tpm = 6


current_position = Position(0,0,0,0)
current_order = None

equity = 0
equity_L = [0]

outputs = []
all_candles = []

cmm = 0.5

@tf.function()
def inference_step(m15_np, m5_np, m1_np, pda_np):
    return model([
        m15_np,
        m5_np,
        m1_np,
        pda_np
    ])



for index in tqdm(range(len(candles))):
#for index in tqdm(range(11000)):
    step(index)
    if( index % 100 == 0 ):
        print("\n", equity_L[-1])

print(equity_L[-1])

  0%|          | 0/20000 [00:00<?, ?it/s]


 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0

 0


 33%|███▎      | 6668/20000 [00:01<00:02, 5866.34it/s]


 0.0

 0.0

 17.25

 76.25


 35%|███▌      | 7044/20000 [00:20<00:50, 254.56it/s] 

...

 39%|███▊      | 7704/20000 [00:53<09:20, 21.94it/s]


 -31.0


 39%|███▉      | 7803/20000 [00:58<10:26, 19.48it/s]


 -31.0


 39%|███▉      | 7842/20000 [01:00<10:15, 19.76it/s]

In [ ]:
def plot_candles(candles):
    for index in range(len(candles)):
        candle = candles[index]
        c = "green" if candle.c > candle.o else "black"
        plt.plot([index, index], [candle.l, candle.h], linewidth=1, color = "black")
        plt.plot([index, index], [candle.c, candle.o], linewidth=3, color = c)
index+=1
#step(index)
plot_candles(m.m1_candles)
if(current_position.direction != 0):
    plt.axhline(current_position.entry, color = "g" if current_position.direction == 1 else "r")
    plt.axhline(current_position.sl, color = "orange")
    plt.axhline(current_position.tp, color = "orange")
if(current_order != None):
    plt.axhline(current_order.entry, color = "g" if current_order.direction == 1 else "r")
    plt.axhline(current_order.sl, color = "orange")
    plt.axhline(current_order.tp, color = "orange")
print(current_position.direction, equity_L[-1])

In [ ]:
#plt.plot(outputs)
plt.plot([x[0] for x in outputs[-400:]], color="r")
plt.plot([x[1] for x in outputs[-400:]], color="g")
plt.plot([x[2] for x in outputs[-400:]], color="b")

In [ ]:
plt.plot([x.c for x in all_candles])


In [ ]:
plt.plot(equity_L)


In [ ]:
plt.plot([x[1]-x[0] for x in outputs], color="b")